In [2]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import numpy
import requests
import pymongo


In [3]:
url = "https://docs.google.com/spreadsheets/d/1zLNAuRqPauss00HDz4XbTH2HqsCzMe0pR8QmD1K8jk8/pubhtml/sheet?headers=false&chrome=false&gid=0&range=A3:I437"

In [4]:
df = pd.read_html(url)

In [5]:
df = df[0]

In [6]:
df.rename(inplace=True, columns={1:'District', 5:'2016', 7:'2012', 9:'2008'})

In [7]:
df = df[["District", "2016", "2012", "2008"]]
df.head()

,District,2016,2012,2008
0,AK-AL,52.8,55.3,59.7
1,AL-01,63.5,61.8,60.9
2,AL-02,64.9,62.9,64.5
3,AL-03,65.3,62.3,62.6
4,AL-04,80.4,74.8,73.3


In [8]:
def fixDistricts(district):
    state = district[0:2]
    distStr = district[3:5]
    if (distStr == 'AL'):
        distNum = 0
    else: distNum = int(distStr)
    newDist = state + '-' + str(distNum)
    return newDist

In [9]:
df['District'] = df['District'].apply(lambda x: fixDistricts(x))
df['geoData'] = ''
df.iloc[434]

District    WY-0
2016        70.1
2012        69.2
2008        65.2
geoData         
Name: 434, dtype: object

In [10]:
for row in numpy.arange(0, len(df)):
    district = df.iloc[row]["District"]
    geoDF = pd.read_json(requests.get('https://theunitedstates.io/districts/cds/2016/{0}/shape.geojson'.format(district)).text)
    geoData = geoDF.loc['coordinates']
    df.iat[row, 4] = geoData[0]

In [11]:
df.head()

,District,2016,2012,2008,geoData
0,AK-0,52.8,55.3,59.7,"[[[[-152.312393188477, 57.36793518066414], [-1..."
1,AL-1,63.5,61.8,60.9,"[[[[-88.03498840332028, 30.522453308105487], [..."
2,AL-2,64.9,62.9,64.5,"[[[[-87.427204, 31.26436], [-87.425511, 31.268..."
3,AL-3,65.3,62.3,62.6,"[[[[-86.577528, 33.801977], [-86.564208, 33.80..."
4,AL-4,80.4,74.8,73.3,"[[[[-88.27005, 33.570819], [-88.269532, 33.572..."


In [12]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [13]:
db = client.electiondata

In [14]:
years = ['2016', '2012', '2008']

for year in years:
    for row in numpy.arange(0,len(df)):
        db[year].insert_one({
            "type": "Feature",
            "properties": {
                "District": df.iloc[row]["District"],
                "Result": df.iloc[row][year],
                "Winner": 'R' if df.iloc[row][year] > 50 else 'D'
            },
            "geometry": {
                "type": "MultiPolygon",
                "coordinates": df.iloc[row]["geoData"]
            }
        })
